# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [55]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [56]:
# checking your current working directory
#print("Curent workspace:", os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*.csv'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [57]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    #print(f)

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [58]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [59]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [60]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [61]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [62]:
# The purpose of the following functions here is just avoid code redundancy and impelements some re-usability as most of the goals with inserting the data is the same.

# Function to write the data on cassandra 
def write_data_to_cassandra(session, query, values):
    """ Writes data to Cassandra, according to a query and values provided. A session must be given as well."""
    try:
        session.execute(query,values)
    except Exception as e:
        print(e)
        
def read_file(file):
    """ Reads the file and return pandas DataFrame. """
    try:
        return(pd.read_csv(file))
    except Exception as e:
        print(e)



In [63]:
# Question 1: Give me the artist, song and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4.
# Creating table song_length_by_session
# The partition key is going to be session_id and the cluster key is going to be item_in_session.
# The purpose here is to make sure that every song listened on every session is uniquely identified
query = "CREATE TABLE IF NOT EXISTS song_length_by_session \
         (session_id int, item_in_session int, artist_name text, song_title text, length decimal, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [64]:
# Question 1: Give me the artist, song and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4.
# Reading data and inserting on the database

# File that contains the events to be read
file = 'event_datafile_new.csv'

# Opening the file
data = read_file(file)

# Creating a query
query = "INSERT INTO song_length_by_session (session_id, item_in_session, artist_name, song_title, length) \
                 VALUES (%s, %s, %s, %s, %s)"

# Inserting the data
for i, row in data.iterrows():
    values = (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length'])
    write_data_to_cassandra(session, query, values)

#### Do a SELECT to verify that the data have been inserted into each table

In [65]:
# Question 1: Give me the artist, song and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4.
# Querying the database and checking the results.
query = "SELECT artist_name, song_title, length \
           FROM song_length_by_session \
          WHERE session_id = 338 \
            AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Iterating the rows and printing the values
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [66]:
# Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# Creating table artists_and_songs_by_session
# The partition key is going to be user_ud and the cluster key will be compound by session_id and item_in_session.
# The purpose here is to make sure that every song listened for every user is unique stored, what is assessed by the item_in_session variable.
query = "CREATE TABLE IF NOT EXISTS artists_and_songs_by_session \
         (user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY(user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [67]:
# Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# Reading data and inserting on the database

# File that contains the events to be read
file = 'event_datafile_new.csv'

# Opening the file
data = read_file(file)

# Creating a query
query = "INSERT INTO artists_and_songs_by_session (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) \
         VALUES (%s, %s, %s, %s, %s, %s, %s)"

# Inserting the data
for i, row in data.iterrows():
    values = (row['userId'], row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['firstName'], row['lastName'])
    write_data_to_cassandra(session, query, values)

In [68]:
# Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# Querying the database and checking the results.
query = "SELECT artist_name, song_title, first_name, last_name \
           FROM artists_and_songs_by_session \
          WHERE user_id = 10 \
            AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Iterating the rows and printing the values
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [69]:
# Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Creating table users_by_songs
# The partition key is going to be song_title and the cluster key will be artist_name and user_id.
# The purpose here is to not to make sure that every time the user listened a song we will have a record but just to record whether the user listened or not a song. That's what the question is asking to. 
# User ID is unique across users whilst first and last name may eventually be duplicated.
query = "CREATE TABLE IF NOT EXISTS users_by_songs "
query = query + "(song_title text, artist_name text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, artist_name, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [70]:
# Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Reading data and inserting on the database

# File that contains the events to be read
file = 'event_datafile_new.csv'

# Opening the file
data = read_file(file)

# Creating a query
query = "INSERT INTO users_by_songs (song_title, artist_name, user_id, first_name, last_name) \
         VALUES (%s, %s, %s, %s, %s)"

# Inserting the data
for i, row in data.iterrows():
    values = (row['song'], row['artist'], row['userId'], row['firstName'], row['lastName'])
    write_data_to_cassandra(session, query, values)

In [71]:
# Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Querying the database and checking the results.
query = "SELECT first_name, last_name \
           FROM users_by_songs \
          WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Iterating the rows and printing the values    
for row in rows:
    print('{} {}'.format(row.first_name,row.last_name))

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [72]:
# Dropping the tables from the database
try:
    session.execute("DROP TABLE IF EXISTS song_length_by_session")
    session.execute("DROP TABLE IF EXISTS artists_and_songs_by_session")
    session.execute("DROP TABLE IF EXISTS users_by_songs")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [73]:
# Closing session and cluster
session.shutdown()
cluster.shutdown()